In [14]:
import os
import logging
import re
import pandas as pd 

from binance_sdk_derivatives_trading_usds_futures.derivatives_trading_usds_futures import (
    DerivativesTradingUsdsFutures,
    ConfigurationRestAPI,
    DERIVATIVES_TRADING_USDS_FUTURES_REST_API_PROD_URL,
)
from binance_sdk_derivatives_trading_usds_futures.rest_api.models import (
    KlineCandlestickDataIntervalEnum,
    TopTraderLongShortRatioPositionsPeriodEnum
)


# Configure logging
logging.basicConfig(level=logging.INFO)

# Create configuration for the REST API
configuration_rest_api = ConfigurationRestAPI(
    api_key="66aLkztQkQFuyUQuM83oqYx9ENuZH1SFFbuTSP3ClcGYa9Vqwy5cmQan9QfK7Gzd",
    # api_key= "O0ambiAxVnDs0pDAtlwVs3uiLR7uH2kh5B12WWWmEersKD9uHipkJZ80I90enw4o",
    # api_secret="",
    api_secret="V4lTk3PUCa7jwGAxPybBFzE7fN2Ob09FZR9BbchLOiGtssRnH9cnkfL2O8C7aoQW",
    base_path=os.getenv(
        "BASE_PATH",  DERIVATIVES_TRADING_USDS_FUTURES_REST_API_PROD_URL)
)

# Initialize DerivativesTradingUsdsFutures client
client = DerivativesTradingUsdsFutures(config_rest_api=configuration_rest_api)

def ticker24hr_price_change_statistics():
    try:
        response = client.rest_api.ticker24hr_price_change_statistics()

        rate_limits = response.rate_limits
        logging.info(f"ticker24hr_price_change_statistics() rate limits: {rate_limits}")

        data = response.data()
        for t in data:
            if t[0] == "actual_instance":    
                return t[1]
        # return data
        # logging.info(f"ticker24hr_price_change_statistics() response: {data}")
    except Exception as e:
        logging.error(f"ticker24hr_price_change_statistics() error: {e}")

#
def in_exchange_trading_symbols():
    try:
        response = client.rest_api.exchange_information()

        rate_limits = response.rate_limits
        # logging.info(f"exchange_info() rate limits: {rate_limits}")

        data = response.data()
        pattern =  r"usdt$"
        usdt_symbols= [
            t.symbol for t in data.symbols
            if re.search(pattern, t.symbol, flags=re.IGNORECASE) and t.status=='TRADING'
        ]
        return usdt_symbols
        logging.info(f"exchange_info() response: {data}")
    except Exception as e:
        logging.error(f"exchange_info() error: {e}")

def sort_tickers():
    """按照涨幅降序排序"""
    tickers = ticker24hr_price_change_statistics()

    if not tickers:
        return []

    in_trading_symbols = in_exchange_trading_symbols()

    if not in_trading_symbols:
        return []


    pattern =  r"usdt$"
    
    usdt_tickers= [
        t for t in tickers
        if re.search(pattern, t.symbol, flags=re.IGNORECASE)
    ]

    in_trading_tickers = [
        t for t in usdt_tickers if t.symbol in in_trading_symbols
    ]
    
    valid_tickers = [
        t for t in  in_trading_tickers
        if t.price_change_percent and not t.symbol.endswith(("UP", "DOWN", "USDTM"))  # 排除杠杆/合约交易对
    ]
    
    sorted_tickers = sorted(
        valid_tickers,
        key=lambda x: float(x.price_change_percent),
        reverse=True
    )
    
    return sorted_tickers

def get_top3_gainers():
    tickers= sort_tickers()
    tickers_list = [vars(ticker) for ticker in tickers[:3]]

    df = pd.DataFrame(tickers_list)
    df['open_time'] = pd.to_datetime(df['open_time'], unit='ms', utc=True).dt.tz_localize(None)
    df['close_time'] = pd.to_datetime(df['close_time'], unit='ms', utc=True).dt.tz_localize(None)
    # 同时优化数值列和时间列
    numeric_columns = ['price_change', 'price_change_percent', 'last_price', 'open_price', 'volume']

    # 数值列转换为浮点数
    for col in numeric_columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')


    return df

In [15]:
tickers = ticker24hr_price_change_statistics()

INFO:root:ticker24hr_price_change_statistics() rate limits: [RateLimit(rateLimitType='REQUEST_WEIGHT', interval='MINUTE', intervalNum=1, count=66, retryAfter=None)]


In [16]:
tickers

[Ticker24hrPriceChangeStatisticsResponse2(symbol='ZRCUSDT', price_change='0.0000180', price_change_percent='0.448', weighted_avg_price='0.0040225', last_price='0.0040350', last_qty='1240', open_price='0.0040170', high_price='0.0040990', low_price='0.0039540', volume='237936767', quote_volume='957112.4739650', open_time=1767437040000, close_time=1767523445002, first_id=10393016, last_id=10421439, count=28422, additional_properties={}),
 Ticker24hrPriceChangeStatisticsResponse2(symbol='AVAAIUSDT', price_change='0.0006100', price_change_percent='6.028', weighted_avg_price='0.0106330', last_price='0.0107300', last_qty='2235', open_price='0.0101200', high_price='0.0110600', low_price='0.0100200', volume='210169615', quote_volume='2234739.7465800', open_time=1767436980000, close_time=1767523421125, first_id=77781173, last_id=77806725, count=25551, additional_properties={}),
 Ticker24hrPriceChangeStatisticsResponse2(symbol='SQDUSDT', price_change='-0.0050400', price_change_percent='-5.944', w

In [17]:
top3 = get_top3_gainers()

INFO:root:ticker24hr_price_change_statistics() rate limits: [RateLimit(rateLimitType='REQUEST_WEIGHT', interval='MINUTE', intervalNum=1, count=106, retryAfter=None)]


In [13]:
top3

,symbol,price_change,price_change_percent,weighted_avg_price,last_price,last_qty,open_price,high_price,low_price,volume,quote_volume,open_time,close_time,first_id,last_id,count,additional_properties
0,MYXUSDT,2.2980,58.369,6.0415584,6.2350,3,3.9370,7.3400000,3.9370000,1.075866e+08,649990900.6605000,2026-01-03 00:48:00,2026-01-04 00:48:51.165,369342640,374332871,4989383,{}
1,BUSDT,0.0772,55.740,0.2033720,0.2157,42,0.1385,0.2350000,0.1372000,1.908637e+09,388163445.4725000,2026-01-03 00:48:00,2026-01-04 00:48:51.339,56502114,59444465,2938837,{}
2,CVXUSDT,0.6390,36.894,2.250211,2.3710,33.9,1.7320,2.444000,1.682000,8.680159e+07,195321853.273900,2026-01-03 00:48:00,2026-01-04 00:48:50.455,10102362,11389978,1286698,{}


In [17]:

pd.DataFrame(top3_list)

,symbol,price_change,price_change_percent,weighted_avg_price,last_price,last_qty,open_price,high_price,low_price,volume,quote_volume,open_time,close_time,first_id,last_id,count,additional_properties
0,HOLOUSDT,0.0244400,36.941,0.0842973,0.0906000,311,0.0661600,0.0921600,0.0655200,1902681265,160390947.2282800,1767269400000,1767355849044,58051664,59408293,1356305,{}
1,RIVERUSDT,2.669000,26.549,12.390115,12.722000,21.2,10.053000,15.400000,9.108000,104005633.0,1288641720.541900,1767269400000,1767355851666,90601908,104588232,13980953,{}
2,1000PEPEUSDT,0.0010817,26.008,0.0048966,0.0052408,20137,0.0041591,0.0053283,0.0041441,227216625107,1112586492.5312649,1767269400000,1767355849445,2376666703,2381205899,4532130,{}


In [18]:
if __name__ == "__main__":
    tickers = get_top3_gainers()
    print("Top 3 Gainers:")
    for t in tickers[:3]:
        print(f"Symbol: {t.symbol}, Change: {t.price_change_percent}%, Open: {t.open_price}, Close: {t.last_price}")
    print("Low 3 Gainers:")
    for t in tickers[-3:]:
        print(f"Symbol: {t.symbol}, Change: {t.price_change_percent}%, Open: {t.open_price}, Close: {t.last_price}")

INFO:root:ticker24hr_price_change_statistics() rate limits: [RateLimit(rateLimitType='REQUEST_WEIGHT', interval='MINUTE', intervalNum=1, count=46, retryAfter=None)]


Top 3 Gainers:


AttributeError: 'str' object has no attribute 'symbol'

In [ ]:
tickers[]